<a href="https://colab.research.google.com/github/tushar2013/data_science_portfolio/blob/master/intro_to_convolutional_nn_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving cats_and_dogs.zip to cats_and_dogs.zip


In [3]:
import zipfile
import os

# Extract uploaded zip
with zipfile.ZipFile("cats_and_dogs.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

# Verify structure
import shutil

# Create Keras-style structure
base_dir = "cats_dogs_data"
os.makedirs(base_dir + "/train/cats", exist_ok=True)
os.makedirs(base_dir + "/train/dogs", exist_ok=True)
os.makedirs(base_dir + "/val/cats", exist_ok=True)
os.makedirs(base_dir + "/val/dogs", exist_ok=True)

# Assuming extracted files are in data/cats and data/dogs
cat_dir = "data/cats"
dog_dir = "data/dogs"

cat_images = os.listdir(cat_dir)
dog_images = os.listdir(dog_dir)

# Split 80-20 train-val
split_ratio = 0.8
split_cat = int(len(cat_images) * split_ratio)
split_dog = int(len(dog_images) * split_ratio)

for i, fname in enumerate(cat_images):
    src = os.path.join(cat_dir, fname)
    dst = os.path.join(base_dir, "train/cats" if i < split_cat else "val/cats", fname)
    shutil.copy(src, dst)

for i, fname in enumerate(dog_images):
    src = os.path.join(dog_dir, fname)
    dst = os.path.join(base_dir, "train/dogs" if i < split_dog else "val/dogs", fname)
    shutil.copy(src, dst)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    base_dir + "/train",
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    base_dir + "/val",
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 0.5065 - loss: 0.6936 - val_accuracy: 0.5025 - val_loss: 0.6898
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.5311 - loss: 0.6907 - val_accuracy: 0.5025 - val_loss: 0.6901
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.5471 - loss: 0.6879 - val_accuracy: 0.5325 - val_loss: 0.6863
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.5449 - loss: 0.6851 - val_accuracy: 0.5725 - val_loss: 0.6832
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.5386 - loss: 0.6858 - val_accuracy: 0.5000 - val_loss: 0.7056
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - accuracy: 0.5457 - loss: 0.6893 - val_accuracy: 0.5800 - val_loss: 0.6798
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - accuracy: 0.5545 - loss: 0.6806 - val_accuracy: 0.5175 - val_loss: 0.6912
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.5710 - loss: 0.6823 - val_accuracy: 0.5825 - val_los

In [7]:
loss, acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {acc*100:.2f}%")

13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 466ms/step - accuracy: 0.6099 - loss: 0.6694
Validation Accuracy: 61.50%


In [11]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]
    label = "Dog" if prediction > 0.5 else "Cat"
    print(f"Prediction: {label} ({prediction:.2f})")

# Upload and test
from google.colab import files
uploaded = files.upload()

# Predict first uploaded file
predict_image(list(uploaded.keys())[0])

Saving dog_new_1.jpg to dog_new_1 (2).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Prediction: Cat (0.47)
